In [5]:
from platform import python_version
import sys

print(sys.executable)
print(python_version())

/Users/chrispap/Documents/EventShapes/env.nosync/bin/python
3.9.5


In [6]:
import numpy as np
import awkward as ak
import uproot
from coffea.nanoevents import NanoEventsFactory, TreeMakerSchema, BaseSchema
from coffea import hist, processor
from coffea.nanoevents.methods import candidate
import matplotlib.pyplot as plt
import mplhep

plt.style.use(mplhep.style.ROOT)
ak.behavior.update(candidate.behavior)

In [7]:
def sphericityTensor(particles):
    particles_p = np.sqrt(particles.x * particles.x + particles.y * particles.y + particles.z * particles.z)
    norm = ak.sum(particles_p * particles_p, axis=1)
    s = np.array([[
                   ak.sum(particles.x * particles.x, axis=1)/norm,
                   ak.sum(particles.x * particles.y, axis=1)/norm,
                   ak.sum(particles.x * particles.z, axis=1)/norm
                  ],
                  [
                   ak.sum(particles.y * particles.x, axis=1)/norm,
                   ak.sum(particles.y * particles.y, axis=1)/norm,
                   ak.sum(particles.y * particles.z, axis=1)/norm
                  ],
                  [
                   ak.sum(particles.z * particles.x, axis=1)/norm,
                   ak.sum(particles.z * particles.y, axis=1)/norm,
                   ak.sum(particles.z * particles.z, axis=1)/norm
                  ]])
        
    return s

def sphericity(s):
    #np.nan_to_num(s)
    s_eigvalues = np.linalg.eigvals(np.moveaxis(s, 2, 0))
    sphericity = 1.5*(s_eigvalues[:,1]+s_eigvalues[:,2])
    return sphericity

In [10]:
fname = '/Users/chrispap/QCD/ak15/QCD/Autumn18.QCD_HT1000to1500_TuneCP5_13TeV-madgraphMLM-pythia8_0_RA2AnalysisTree.root'
events = NanoEventsFactory.from_root(fname, treepath='TreeMaker2/PreSelection', schemaclass=TreeMakerSchema, entry_stop=1000).events()

histo = hist.Hist("Events",
          hist.Bin("nTracks", "multiplicity", 50, 0, 250),
          hist.Bin("sph", "sphericity", 50, 0, 1),
         )

integratedLuminosity = 137.19*1000 # fb^{-1} to pb^{-1}
        
ht = events.HT
weights = integratedLuminosity*events.CrossSection[ht > 1200]/len(events)
tracks = events.Tracks
tracks_pt = np.sqrt(tracks.x**2 + tracks.y**2)
tracks_eta = np.arcsinh(tracks.z / tracks_pt)
finalTracks = (tracks_pt > 1) & (abs(tracks_eta) < 2.5) & (tracks.fromPV0 >= 2) & tracks.matchedToPFCandidate
nTracks = ak.sum(finalTracks[ht > 1200], axis=1)
sTensor = sphericityTensor(tracks[finalTracks][ht > 1200])
sph = sphericity(sTensor)

histo.fill(
    nTracks=nTracks,
    sph=sph,
    weight=weights
)

In [11]:
nTracks

<Array [37, 69, 76, 51, 67, ... 73, 68, 80, 45] type='282 * int64'>

In [12]:
sph

array([4.95290815e-02, 3.67008257e-01, 2.56385742e-01, 4.13174721e-01,
       5.97208034e-02, 8.25874883e-02, 5.88551096e-01, 3.71516542e-03,
       1.62790887e-01, 6.56628779e-02, 1.48860517e+00, 1.44726659e+00,
       1.39000639e+00, 1.12426805e-01, 1.55684329e-01, 7.20464082e-02,
       1.09463032e-01, 1.31065902e-01, 5.18767631e-02, 3.99039193e-02,
       1.46518660e+00, 9.44926571e-02, 1.49959049e+00, 1.48246799e+00,
       1.22146227e-01, 2.12699095e-02, 1.11535353e-01, 2.08969337e-01,
       1.46402704e-02, 1.09254579e-01, 3.46201579e-01, 7.32774414e-02,
       3.60551415e-01, 3.16811415e-01, 1.49620131e+00, 3.31310970e-01,
       2.75749682e-01, 1.49796914e+00, 2.04553615e-01, 3.36892511e-01,
       5.51779018e-02, 2.75444298e-01, 1.41644342e+00, 1.19723202e-01,
       1.54676318e-01, 1.44585905e+00, 3.03400440e-01, 7.37257327e-02,
       6.80978072e-02, 1.47680676e+00, 3.43315126e-01, 2.32193220e-01,
       1.48191077e-01, 1.31768998e-02, 8.04385230e-04, 3.59823633e-01,
      